In [6]:
import os
import re
from pathlib import Path
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [8]:
def is_gzip_file(filepath):
    with open(filepath, 'rb') as f:
        return f.read(2) == b'\x1f\x8b'
def display_all(df):
    plot_cols = df.columns
    plot_features = df[plot_cols]
    plot_features.index = df.index
    _ = plot_features.plot(subplots=True)

    plot_features = df[plot_cols][:960]
    plot_features.index = df.index[:960]
    _ = plot_features.plot(subplots=True)

In [32]:
csv_main = tf.keras.utils.get_file(
    origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/device-1_filled.csv.zip')
    #origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/utci_selected_timeseries_filled_15min.csv.zip')
    #origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/era5-hourly_timeseries_filled_15min.csv.zip')
    #origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/era5-land_timeseries_filled_15min.csv.zip')
csv_main = '/Users/watcharinsarachai/workpaces/PhotovoltaicHA/src/jupyter_nb/merge_15min.csv'
csv_main

'/Users/watcharinsarachai/workpaces/PhotovoltaicHA/src/jupyter_nb/merge_15min.csv'

In [48]:
csv_ext = tf.keras.utils.get_file(
    #origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/data_1_filled.csv.tar.gz')
    #origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/utci_selected_timeseries_filled_15min.csv.zip')
    #origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/era5-hourly_timeseries_filled_15min.csv.zip')
    origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/era5-land_timeseries_filled_15min.csv.zip')
csv_ext

3741592/3741592 ━━━━━━━━━━━━━━━━━━━━ 4s 1us/step


'/Users/watcharinsarachai/.keras/datasets/era5-land_timeseries_filled_15min.csv.zip'

In [49]:
date_col = 'datetime' 
if is_gzip_file(csv_main):
  # Read the data directly into a pandas DataFrame
  df_main = pd.read_csv(csv_main, compression='gzip')
else:
  # Read the data without compression
  df_main = pd.read_csv(csv_main)

if is_gzip_file(csv_ext):
  # Read the data directly into a pandas DataFrame
  df_ext = pd.read_csv(csv_ext, compression='gzip')
else:
  # Read the data without compression
  df_ext = pd.read_csv(csv_ext)

In [51]:
df_main = df_main.rename(columns={df_main.columns[0]: date_col})
df_main.head(5)

,datetime,ambient_temperature,current_power,current_value_of_consumption,external_energy_supply,grid_feed_in,internal_power_supply,self_consumption,temperature_measurement,total_irradiation,utci_mean,cc,q,r,t,u,v
0,2021-11-06 11:15:00,30.288889,46519.355556,37280.422222,0.0,9238.933333,37280.422222,37280.422222,58.022222,798.444444,302.213731,0.0,0.011205,63.667006,293.487773,0.555862,-1.168195
1,2021-11-06 11:30:00,30.076923,47600.714286,39472.626374,0.0,9397.756098,39132.406593,39132.406593,58.263736,812.571429,302.213731,0.0,0.011205,63.667006,293.487773,0.555862,-1.168195
2,2021-11-06 11:45:00,30.629213,47056.988764,32669.393258,0.0,14387.595506,32669.393258,32669.393258,57.235955,807.224719,302.213731,0.0,0.011205,63.667006,293.487773,0.555862,-1.168195
3,2021-11-06 12:00:00,31.021978,48415.835165,36126.516484,0.0,12289.318681,36126.516484,36126.516484,56.846154,817.527473,299.223424,0.0,0.011366,65.189689,293.320821,0.497185,-0.620988
4,2021-11-06 12:15:00,31.831461,49345.404494,36879.966292,0.0,12465.438202,36879.966292,36879.966292,55.842697,816.651685,299.223424,0.0,0.011366,65.189689,293.320821,0.497185,-0.620988


In [52]:
df_ext = df_ext.rename(columns={df_ext.columns[0]: date_col})
df_ext.head(5)

,datetime,t2m,fal,slhf,ssr,str,sshf,ssrd,strd,u10,v10,sp,tp
0,2021-12-31 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-12-31 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-12-31 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-12-31 01:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-12-31 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# Ensure timestamp column is datetime type
if df_main[date_col].dtype != 'datetime64[ns]':
    df_main[date_col] = pd.to_datetime(df_main[date_col])
if df_ext[date_col].dtype != 'datetime64[ns]':
    df_ext[date_col] = pd.to_datetime(df_ext[date_col])

In [54]:
print(f"Main total rows: {df_main.shape[0]}")
print(f"Ext total rows: {df_ext.shape[0]}")

Main total rows: 138238
Ext total rows: 133917


In [55]:
print(df_main[date_col].dt.year.unique())
print(df_main[date_col].dt.year.value_counts())
print(df_main.shape)

[2021 2022 2023 2024 2025]
datetime
2024    35136
2022    35040
2023    35040
2025    27691
2021     5331
Name: count, dtype: int64
(138238, 17)


In [56]:
# Sort by time if it exists
df_main.index = df_main[date_col]
df_main.pop(date_col)
df_main = df_main.sort_index()
print(df_main.shape)

(138238, 16)


In [57]:
# Prepare ext: set index and sort
df_ext.index = df_ext[date_col]
df_ext.pop(date_col)
df_ext = df_ext.sort_index()
print(df_ext.shape)

(133917, 12)


In [58]:
# Diagnose and fix index issues
print(f"Index dtype: {df_main.index.dtype}")
print(f"Has NaT: {df_main.index.isna().sum()}")
print(f"Has duplicates: {df_main.index.duplicated().sum()}")
print(f"Is monotonic: {df_main.index.is_monotonic_increasing}")

Index dtype: datetime64[ns]
Has NaT: 0
Has duplicates: 0
Is monotonic: True


In [59]:
# Join only difference: add ext columns that don't exist in main, aligned by time
unique_ext_cols = df_ext.columns.difference(df_main.columns)
print(f"Unique ext columns to add: {list(unique_ext_cols)}")

Unique ext columns to add: ['fal', 'slhf', 'sp', 'sshf', 'ssr', 'ssrd', 'str', 'strd', 't2m', 'tp', 'u10', 'v10']


In [60]:
df_merged = df_main.join(df_ext[unique_ext_cols], how='left')
print(f"Merged shape: {df_merged.shape}")

Merged shape: (138238, 28)


In [61]:
# Compressed CSV
df_merged.to_csv('merge_15min.csv')